In [1]:
import chromadb

client = chromadb.Client()
collection = client.create_collection(name="my_collection")

collection.add(
    documents=[
        "This document is about New York",
        "This document is about Delhi"
    ],
    ids= ['id1','id2']
)

all_docs = collection.get()
all_docs

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'documents': ['This document is about New York',
  'This document is about Delhi'],
 'uris': None,
 'data': None,
 'metadatas': [None, None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [2]:
from langchain_groq import ChatGroq

llm=ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.3-70b-versatile"
)
response=llm.invoke("first person to walk on moon was...")
print(response.content)

The first person to walk on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/110690555461018310-software-engineer-iii-infrastructure-core")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Software Engineer III, Infrastructure, Core — Google CareersCareersSkip navigation linksTeamsLocationsBenefitsJobsStudentsYour careerarrow_drop_downYour career menu ApplicationsProfileSaved jobsJob alertsSign inCareersApplicationsProfileSearch jobsSaved jobsJob alertsTeamsLocationsBenefitsJobsStudentsHelpSend Feedbackjob detailsarrow_backBack to jobs searchJobs search results2,815  jobs matchedAccount Executive, Mid-Market Sales, Google Customer SolutionsMumbai, Maharashtra, India; Gurgaon, Haryana, India; +2 more; +1 moreSoftware Engineer III, Infrastructure, CoreBengaluru, Karnataka, IndiaSenior Software Engineer, AI/ML GenAI, Google Cloud AISunnyvale, CA, USA; Kirkland, WA, USASenior Programmatic SpecialistHyderabad, Telangana, India; Gurugram, Haryana, IndiaField Solutions Architect, GenAI, Google Cloud (French)Paris, FranceSite Facilities Operations Manager (Thai, English)Bangkok, ThailandNetwork Implementation Engineer, Production Network ImplementationCouncil Bluffs, IA, USAAsso

In [4]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job posting and return them in JSON format containing the 
    following keys: 'role', 'experience', 'skills' and 'description'.
    only return the valid JSON
    ### VALID JSON (NO PREAMBLE):
    """
    #no preamble means no initial content of data just main content
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})

print(res.content)

```json
{
  "role": "Software Engineer III, Infrastructure, Core",
  "experience": "2 years of experience with software development in one or more programming languages, or 1 year of experience with an advanced degree. 2 years of experience with data structures or algorithms in either an academic or industry setting.",
  "skills": [
    "Programming languages",
    "Data structures",
    "Algorithms",
    "Infrastructure",
    "Distributed systems",
    "Networking",
    "Compute technologies",
    "Storage",
    "Hardware architecture"
  ],
  "description": "Google's software engineers develop the next-generation technologies that change how billions of users connect, explore, and interact with information and one another. The Core team builds the technical foundation behind Google’s flagship products."
}
```


In [5]:
type(res.content)

str

In [6]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer III, Infrastructure, Core',
 'experience': '2 years of experience with software development in one or more programming languages, or 1 year of experience with an advanced degree. 2 years of experience with data structures or algorithms in either an academic or industry setting.',
 'skills': ['Programming languages',
  'Data structures',
  'Algorithms',
  'Infrastructure',
  'Distributed systems',
  'Networking',
  'Compute technologies',
  'Storage',
  'Hardware architecture'],
 'description': "Google's software engineers develop the next-generation technologies that change how billions of users connect, explore, and interact with information and one another. The Core team builds the technical foundation behind Google’s flagship products."}

In [7]:
type(json_res)

dict

In [9]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [10]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [14]:
links = collection.query(query_texts=["Experience in Python","Expertise in React Native"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [16]:
job = json_res
job['skills']

['Programming languages',
 'Data structures',
 'Algorithms',
 'Infrastructure',
 'Distributed systems',
 'Networking',
 'Compute technologies',
 'Storage',
 'Hardware architecture']

In [17]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Solutions for Google's Core Team

Dear Hiring Manager,

I came across the job description for a Software Engineer III, Infrastructure, Core at Google, and I am excited to introduce AtliQ, an AI & Software Consulting company that can fulfill your requirements. With our expertise in software development, data structures, and algorithms, we are confident that we can provide the necessary support to build the technical foundation behind Google's flagship products.

At AtliQ, we have a team of skilled software engineers with experience in programming languages, infrastructure, distributed systems, networking, compute technologies, storage, and hardware architecture. Our team has a proven track record of delivering scalable, optimized, and efficient solutions that cater to the needs of our clients. We have empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.